## Without threads

In [1]:
import time 
start = time.perf_counter()
def do_something(time_to_run = 1):
    print(f'Sleeping {time_to_run} sec...')
    time.sleep(time_to_run)
    print('Done sleeping')

do_something()
do_something()
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1 sec...
Done sleeping
Sleeping 1 sec...
Done sleeping
Finished in 2.01 second(s)


## With threads

In [3]:
import threading 

start = time.perf_counter()
t1 = threading.Thread(target = do_something,args = [1.5])
t2 = threading.Thread(target = do_something,args = [1.5])
t1.start()
t2.start()

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1.5 sec...
Sleeping 1.5 sec...Finished in 0.01 second(s)

Done sleeping
Done sleeping


## Waiting for threads to join

In [5]:
start = time.perf_counter()
t1 = threading.Thread(target = do_something ,args = [1.5] )
t2 = threading.Thread(target = do_something ,args = [1.5] )
t1.start()
t2.start()
t1.join()
t2.join()
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1.5 sec...
Sleeping 1.5 sec...
Done sleepingDone sleeping

Finished in 1.52 second(s)


In [8]:
start = time.perf_counter()
threads = []
for _ in range(10):
    t = threading.Thread(target = do_something ,args = [1])
    t.start()
    threads.append(t)
for thread in threads:
    thread.join()
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Done sleepingDone sleeping
Done sleeping
Done sleeping
Done sleepingDone sleeping


Done sleeping
Done sleepingDone sleeping

Done sleeping
Finished in 1.03 second(s)


## Concurrent futures

In [23]:
import concurrent.futures
start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    for _ in range(10):
        executor.submit(do_something, 1)

# Wait until all submitted processes complete
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Done sleeping
Done sleepingDone sleeping
Done sleeping

Done sleeping
Done sleeping
Done sleeping
Done sleeping
Done sleeping
Done sleeping
Finished in 1.03 second(s)


In [24]:
def new_do_something(time_to_run):
    print(f'Sleeping {time_to_run} sec...')
    time.sleep(time_to_run)
    return f'Done sleeping for {time_to_run}'

start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    seconds = [1,2,3,4,5]
    futures = [executor.submit(new_do_something , i) for i in seconds]
    for res in concurrent.futures.as_completed(futures): # return each result as soon as it is completed:
        print(res.result())

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1 sec...
Sleeping 2 sec...
Sleeping 3 sec...
Sleeping 4 sec...
Sleeping 5 sec...
Done sleeping for 1
Done sleeping for 2
Done sleeping for 3
Done sleeping for 4
Done sleeping for 5
Finished in 5.03 second(s)


## Race condition

In [28]:
import logging
import threading
import time
import concurrent.futures

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

19:48:55: Testing update. Starting value is 0.
19:48:55: Thread 0: starting update
19:48:55: Thread 1: starting update
19:48:56: Thread 1: finishing update
19:48:56: Thread 0: finishing update
19:48:56: Testing update. Ending value is 1.


Finished in 1.02 second(s)


## Locks

In [30]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s about to lock", name)
        with self._lock:
            logging.debug("Thread %s has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(1)
            self.value = local_copy
            logging.debug("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)


In [31]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,datefmt="%H:%M:%S")

start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.locked_update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

19:49:18: Testing update. Starting value is 0.
19:49:18: Thread 0: starting update
19:49:18: Thread 1: starting update
19:49:19: Thread 0: finishing update
19:49:20: Thread 1: finishing update
19:49:20: Testing update. Ending value is 2.


Finished in 2.03 second(s)
